In [1]:
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
dataset = pd.read_parquet("../lumen_dataset/train.parquet")
dataset.head()

,reservation_id,night_number,stay_date,guest_id,guest_country_id,reservation_status,reservation_date,date_from,date_to,resort_id,...,price,price_tax,total_price_tax,total_price,food_price,food_price_tax,other_price,other_price_tax,room_category_id,sales_channel_id
0,73710,1.0,2007-12-13,22897,HR,Checked-out,2007-11-28,2007-12-13,2007-12-15,1,...,4255.462,425.517,452.089,4564.690,265.428,26.572,43.800,0.000,3,10.0
1,73710,2.0,2007-12-14,22897,HR,Checked-out,2007-11-28,2007-12-13,2007-12-15,1,...,4243.709,424.349,450.921,4552.937,265.428,26.572,43.800,0.000,3,10.0
2,74464,1.0,2008-01-01,106278,HR,Checked-out,2007-12-29,2008-01-01,2008-01-02,1,...,4336.857,433.693,3806.147,19764.823,530.929,53.071,14897.037,3319.383,4,4.0
3,74461,1.0,2008-01-01,38936,GB,Cancelled,2007-12-29,2008-01-01,2008-01-02,1,...,8536.766,853.662,1012.948,10392.280,1592.714,159.286,262.800,0.000,5,3.0
4,74466,1.0,2008-01-01,106279,HR,Cancelled,2007-12-29,2008-01-01,2008-01-03,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6,4.0


Check the range of dates in the dataset

In [3]:
min_date = dataset['reservation_date'].min()
max_date = dataset['reservation_date'].max()

print(f"Minimum reservation date: {min_date}")
print(f"Maximum reservation date: {max_date}")

Minimum reservation date: 2007-03-04
Maximum reservation date: 2009-12-31


We notice that 2008 is the only year where we have all of the data. Therefore we do our analysis in context of that year, while hypothetizing that the data for rest of the years is similar.
Specifically we count the number of reservations per room type in that exact year. In this way we can know which room types are more important then others.

In [4]:
dataset['reservation_date'] = pd.to_datetime(dataset['reservation_date'])

dataset_2008 = dataset[dataset['reservation_date'].dt.year == 2008]

reservations_per_room_type_2008 = dataset_2008.groupby('room_category_id').size().reset_index(name='num_reservations')

print(reservations_per_room_type_2008)

   room_category_id  num_reservations
0                 1                39
1                 2              5752
2                 3              3466
3                 4               535
4                 5              4451
5                 6              1579
6                 7               167
7                11                40


As we are planing on using the num of reservations per room as the room category id, we need to scale them based on the other values in our dataset. So between 0 and 1

In [5]:
max_reservations_2008 = reservations_per_room_type_2008['num_reservations'].max()

reservations_per_room_type_2008['scaled_room_id'] = round(reservations_per_room_type_2008['num_reservations'] / max_reservations_2008,3)

print(reservations_per_room_type_2008)

   room_category_id  num_reservations  scaled_room_id
0                 1                39           0.007
1                 2              5752           1.000
2                 3              3466           0.603
3                 4               535           0.093
4                 5              4451           0.774
5                 6              1579           0.275
6                 7               167           0.029
7                11                40           0.007


As our initial dataset we will be keeping day of the week, day of the year, the number of reservations for that day and for that specific room category id

In [6]:
dataset['reservation_date'] = pd.to_datetime(dataset['reservation_date'])

dataset['day_of_week'] = dataset['reservation_date'].dt.dayofweek + 1  # +1 to make Monday=1, Sunday=7
dataset['day_of_year'] = dataset['reservation_date'].dt.dayofyear

new_dataset = dataset.groupby(['day_of_week', 'day_of_year', 'room_category_id']).size().reset_index(name='num_reservations')

In [7]:
print(new_dataset.head())

   day_of_week  day_of_year  room_category_id  num_reservations
0            1            5                 3                40
1            1            5                 5                 4
2            1            5                 6                 1
3            1           12                 5                 1
4            1           12                 6                 4


Now let's see how our scaled room ids compare to the original room category ids

In [8]:
merged_dataset = pd.merge(new_dataset, reservations_per_room_type_2008[['room_category_id', 'scaled_room_id']], on='room_category_id', how='left')

print(merged_dataset.head())

   day_of_week  day_of_year  room_category_id  num_reservations  \
0            1            5                 3                40   
1            1            5                 5                 4   
2            1            5                 6                 1   
3            1           12                 5                 1   
4            1           12                 6                 4   

   scaled_room_id  
0           0.603  
1           0.774  
2           0.275  
3           0.774  
4           0.275  


We see that we have merged the dataset in a corect way, so we can then remove the room category id and just keep the scaled room id

In [9]:
dataset_with_normalized_id = merged_dataset.drop(columns=['room_category_id'])

print(dataset_with_normalized_id.head())

   day_of_week  day_of_year  num_reservations  scaled_room_id
0            1            5                40           0.603
1            1            5                 4           0.774
2            1            5                 1           0.275
3            1           12                 1           0.774
4            1           12                 4           0.275
